In [1]:
#Imports
import json
import pandas as pd
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
import boto3
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, HTML
from datetime import datetime, timedelta
import hashlib
from SearchApp import SearchApp
from Cache import TweetCache

In [2]:
search_app = SearchApp()

In [3]:
tweet_cache = TweetCache(max_size=30)

In [4]:
# Define function to handle search
def handle_search(sender):
    start_time = datetime.now()
    keyword = search_input.value
    category = category_dropdown.value
    start_date = start_date_picker.value.strftime('%Y-%m-%d') if start_date_picker.value else None
    end_date = end_date_picker.value.strftime('%Y-%m-%d') if end_date_picker.value else None
    results = search_app.final_search_app(category, keyword, tweet_cache)
    filtered_results = filter_results_by_date(results, start_date, end_date)
    sorted_results = sort_results(filtered_results)
    end_time = datetime.now()
    search_duration = end_time - start_time
    display_results(sorted_results)
    display(HTML(f"<p>Search completed in: {search_duration}</p>"))

# Define function to filter results by date
def filter_results_by_date(results, start_date, end_date):
    if start_date and end_date:
        filtered_results = [tweet for tweet in results if start_date <= tweet.get('time', datetime.now()).strftime('%Y-%m-%d') <= end_date]
        return filtered_results
    else:
        return results

# Define function to sort results
def sort_results(results):
    if sort_dropdown.value == 'Time':
        key = lambda x: x.get('time', datetime.now())
    elif sort_dropdown.value == 'Likes':
        key = lambda x: x.get('favorite_count', 0)
    elif sort_dropdown.value == 'Retweets':
        key = lambda x: x.get('retweet_count', 0)
    elif sort_dropdown.value == 'Quotes':
        key = lambda x: x.get('quote_count', 0)
    elif sort_dropdown.value == 'Replies':
        key = lambda x: x.get('reply_count', 0)
    else:
        key = lambda x: x.get('time', datetime.now())
    sorted_results = sorted(results, key=key, reverse=sort_order_dropdown.value == 'Descending')
    return sorted_results

def display_results(results):
    output_area.clear_output()
    if not results: 
        with output_area: 
            print("No results found.") 
    else: 
        with output_area: 
            for result in results:
                display_tweet(result)


# Define function to display search results

def display_tweet(tweet):
    default_profile_image_url = "https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png"
    user_profile_image_url = tweet.get('user_profile_image_url', default_profile_image_url)
    tweet_time = tweet.get('time', datetime.now()).strftime('%Y-%m-%d')
    tweet_text = tweet_text_without_retweet(tweet['text'])

    # Check if the tweet is a retweet
    is_retweet = tweet_text.startswith("RT @")
    if is_retweet:
        retweeted_user = tweet_text.split(":", 1)[0].split("RT @")[1].strip()
        retweet_prefix = f"Retweeted from @{retweeted_user}"
    else:
        retweet_prefix = ""

    # Generate a unique identifier for the tweet
    tweet_id = hashlib.md5(tweet_text.encode()).hexdigest()

    # Get retweet metadata
    retweet_metadata = tweet.get('retweet_metadata', [])
    
    # Get quote metadata
    quote_metadata = tweet.get('quote_metadata', [])

    tweet_html = f"""
        <div id="tweet_{tweet_id}" style="position: relative; border: 1px solid #ccc; border-radius: 8px; padding: 10px; margin-bottom: 10px;">
            <div style="display: flex; align-items: center; margin-bottom: 10px;">
                <img src="{user_profile_image_url}" style="width: 50px; height: 50px; border-radius: 50%; margin-right: 10px;" onerror="this.src='{default_profile_image_url}'; this.onerror=null;">
                <div>
                    <div><b>{tweet['user_name']}</b> @{tweet['user_screen_name']} {verified_badge(tweet['user_verification'])}</div>
                    <span style="color: #888; font-size: 14px; position: absolute; top: 5px; right: 10px;">{retweet_indicator(tweet['text'])}</span>
                    <div style="color: #888;">{tweet['user_description']}</div>
                    <div style="font-size: 12px; color: #888; float: right;">{retweet_prefix}</div>
                </div>
            </div>
            <p style="margin-bottom: 7px;">{tweet_text}</p>
            <div style="font-size: 12px; color: #888; margin-bottom: 5px;">{tweet_time}</div>
            <div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 10px;">
                <div>
                    <i class="fa fa-comment" style="margin-right: 5px;"></i> {tweet['reply_count']}
                    <i class="fa fa-retweet" style="margin-left: 10px; margin-right: 5px;"></i> {tweet['retweet_count']}
                    <i class="fa fa-heart" style="margin-left: 10px; margin-right: 5px;"></i> {tweet['favorite_count']}
                    <i class="fa fa-quote-right" style="margin-left: 10px; margin-right: 5px;"></i> {tweet['quote_count']}
                </div>
                <a href="{tweet['original_tweet_link']}" target="_blank">Original Tweet</a>
            </div>
            <div id="tweet_details_{tweet_id}" style="display: none; margin-top: 10px;">
                <b><p>Followers: </b>{tweet['user_followers_count']}</p>
                <b><p>Following: </b>{tweet['user_friends_count']}</p>
                <b><p>Website: </b>{tweet['user_profile_url']}</p>
                <p><b><i>Recent Tweets by the User:</i></b></p>
                <ul>
                    {"".join(f"<li>{recent_tweet['created_at'].strftime('%Y-%m-%d')}: {recent_tweet['text']}</li>" for recent_tweet in tweet['ordered_user_tweets'])}
                </ul>
                <p><b><i>Recent Retweets of this Tweet:</i></b></p>
                <ul>
                    {"".join(f"<li>{retweet['retweet_time'].strftime('%Y-%m-%d')}: {retweet['retweet_text']} by {retweet['retweet_user']}</li>" for retweet in retweet_metadata)}
                </ul>
                <p><b><i>Recent Quotes of this Tweet:</i></b></p>
                <ul>
                    {"".join(f"<li>{quote['quote_time'].strftime('%Y-%m-%d')}: {quote['quote_text']} by {quote['quote_user']}</li>" for quote in quote_metadata)}
                </ul>
            </div>
            <div style="cursor: pointer; color: blue; text-decoration: underline;" onclick="toggleDetails('{tweet_id}')">
                <span id="toggle_text_{tweet_id}" style="font-weight: bold;">Show Details</span>
            </div>
        </div>
        <script>
            function toggleDetails(tweetId) {{
                var tweetDetails = document.getElementById('tweet_details_' + tweetId);
                var toggleText = document.getElementById('toggle_text_' + tweetId);
                if (tweetDetails.style.display === 'none') {{
                    tweetDetails.style.display = 'block';
                    toggleText.innerText = 'Hide Details';
                }} else {{
                    tweetDetails.style.display = 'none';
                    toggleText.innerText = 'Show Details';
                }}
            }}
        </script>
    """
    display(HTML(tweet_html))
    
# Define function to generate verified badge
def verified_badge(verified):
    if verified:
        return '<i class="fa fa-check-circle" style="color: blue; margin-left: 5px;"></i>'
    else:
        return ''

# Define function to remove "RT @username:" prefix from tweet text

def tweet_text_without_retweet(tweet_text):
    if tweet_text.startswith("RT @"):
        return tweet_text.split(":", 1)[1].strip()
    else:
        return tweet_text


def retweet_indicator(tweet_text):
    if tweet_text.startswith("RT @"):
        original_account = tweet_text.split(":")[0][4:]
        return f'Retweet from @{original_account}'
    else:
        return ''

# Define function to handle refresh
def handle_refresh(sender):
    search_input.value = ''  # Clear text input field
    category_dropdown.value = None  # Reset dropdown value
    start_date_picker.value = None  # Reset start date picker value
    end_date_picker.value = None  # Reset end date picker value
    sort_dropdown.value = 'Time'  # Reset sort dropdown value
    sort_order_dropdown.value = 'Descending'  # Reset sort order dropdown value
    search_button.disabled = True  # Disable search button
    output_area.clear_output()  # Clear output area

# Define function to update search button state
def update_search_button_state(change):
    if search_input.value and category_dropdown.value and start_date_picker.value and end_date_picker.value:
        search_button.disabled = False
    else:
        search_button.disabled = True

# Create heading
heading = widgets.HTML("<h2>Twitter Search Engine</h2>")

# Create text input widget for search query
search_input = widgets.Text(placeholder="Enter keyword", layout=widgets.Layout(width='500px'))
search_input.observe(update_search_button_state, 'value')

# Create dropdown menu for search category
category_dropdown = widgets.Dropdown(
    options=['User', 'Tweet Text', 'Hashtag', 'User Mention'],
    value=None,
    description='Category:',
    layout=widgets.Layout(width='200px')
)
category_dropdown.observe(update_search_button_state, 'value')

# Create date picker widget for start date
start_date_picker = widgets.DatePicker(
    description='Start Date:',
    layout=widgets.Layout(width='200px')
)
start_date_picker.observe(update_search_button_state, 'value')

# Create date picker widget for end date
end_date_picker = widgets.DatePicker(
    description='End Date:',
    layout=widgets.Layout(width='200px')
)
end_date_picker.observe(update_search_button_state, 'value')

# Create dropdown menu for sorting criteria
sort_dropdown = widgets.Dropdown(
    options=['Time', 'Likes', 'Retweets', 'Quotes', 'Replies'],
    value='Time',
    description='Sort By:',
    layout=widgets.Layout(width='200px')
)

# Create dropdown menu for sort order
sort_order_dropdown = widgets.Dropdown(
    options=['Ascending', 'Descending'],
    value='Descending',
    description='Order:',
    layout=widgets.Layout(width='200px')
)

# Create search button
search_button = widgets.Button(description="Search", button_style="info", icon="search", disabled=True)
search_button.on_click(handle_search)

# Create refresh button
refresh_button = widgets.Button(description="Refresh", button_style="info", icon="refresh")
refresh_button.on_click(handle_refresh)

# Arrange widgets horizontally
form_row1 = widgets.HBox([search_input, category_dropdown, start_date_picker, end_date_picker])
form_row2 = widgets.HBox([search_button, refresh_button, sort_dropdown, sort_order_dropdown])

# Create output area to display search results
output_area = widgets.Output()

# Display the heading, form, output area, and refresh button vertically
display(widgets.VBox([heading, form_row1, form_row2, output_area]))